In [157]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
%matplotlib inline

# If you would like to make further imports from Tensorflow, add them here

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [158]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [159]:
# /content/drive/MyDrive/BankData/bank-full.csv
%cd "/content/drive/MyDrive/BankData"

/content/drive/MyDrive/BankData


**OBTENER EL DATASET**

Cargar mi dataset y dividirlo para lo que usaremos para el test y el train

In [160]:
df = pd.read_csv("bank-full.csv", delimiter=";")

*Limpiar los datos*

Seleccionar columnas que usaremos para el modelo


In [161]:
columnas = ['age', 'marital', 'education', 'default', 'loan', 'duration', 'y']
df_columnas_seleccionadas = df[columnas]

In [162]:
df_columnas_seleccionadas = df_columnas_seleccionadas[df_columnas_seleccionadas['education'] != 'unknown']
df_columnas_seleccionadas.reset_index(drop=True, inplace=True)

**SEPARACIÓN DEL SET DE PRUEBA Y DEL SET DE TRAINING**

Ya que lo limpiamos, ahora si dividimos el data set para tener el conjunto que sera entrenado y el conjunto que sera para pruebas (train_data y test_data)

In [163]:
test_data = df_columnas_seleccionadas.iloc[:21677]
train_data = df_columnas_seleccionadas.iloc[21677:]

Como queremos estudiar si contrataran o no la tarjeta, y ese dato viene en la columna Y, debemos quitar esa información de nuestros fragmentos de datos

In [164]:
test_labels_target = test_data["y"]
test_data = test_data.drop(columns=["y"])
train_labels_target = train_data["y"]
train_data = train_data.drop(columns=["y"])
## modelo de entrenamiento
## punto fit paso matrix de las x y de las Y

**PREPROCESADO DEL TARGET**

Como el modelo solo acepta números, debemos pasar mi target a binario, para eso usaremos Label encoder, ya que:

*   Solo tenemos dos categorias (yes / no)
*   Al tener solo dos categorias, no hay problema confusión de orde
*   Usa menos memoria



In [165]:
enc = LabelEncoder()
train_labels_target = enc.fit_transform(train_labels_target)
test_labels_target = enc.fit_transform(test_labels_target)

**PREPROCESADO DE FEATURES**

Usamos OneHotEncoder ya que tenemos mas de dos categorias, nos guiamos con "Titanic Solution One-Hot-Encode All The Things"

In [166]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_data[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(test_data[object_cols]))

OH_cols_train.columns = OH_encoder.get_feature_names_out()
OH_cols_test.columns = OH_encoder.get_feature_names_out()

# One-hot encoding removed index; put it back
OH_cols_train.index = train_data.index
OH_cols_test.index = test_data.index

# Add one-hot encoded columns to numerical features (age - duration)
numeric_columns = train_data.drop(columns=object_cols).columns
numeric_train = train_data[numeric_columns]
numeric_test = test_data[numeric_columns]

train_data = pd.concat([OH_cols_train, numeric_train], axis=1)
test_data = pd.concat([OH_cols_test, numeric_test], axis=1)

test_data.head()


,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,default_no,default_yes,loan_no,loan_yes,age,duration
0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,58,261
1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,44,151
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,33,76
3,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,35,139
4,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,28,217
